In [2]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import reproducible_comp, get_final_df
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

In [4]:
file_path = "../../../data/processed_data/force_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_inner, category_force_inner, category_names, df_force_inner = get_final_df(data, is_acc_data=0, is_multilabel=1)

file_path = "../../../data/processed_data/force_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_outer, category_force_outer, category_names, df_force_outer = get_final_df(data, is_acc_data=0, is_multilabel=1)

merged_features_force = np.concatenate((features_force_inner,features_force_outer), axis=0)
merged_category_force = np.concatenate((category_force_inner,category_force_outer), axis=0)

labels = merged_category_force.copy()
features=merged_features_force.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (2680, 1)
Labels Shape:  (2680, 4)


In [5]:
max_length = 150
for i in range (len(features)):
    if(len(features[i][0]<max_length)):
        features[i][0]=np.pad(features[i][0], (0,max_length-len(features[i][0])), 'constant', constant_values=(0))
feat= np.zeros(len(features) *max_length).reshape(-1, max_length)
for i in range(len(features)):
    feat[i]= np.concatenate((features[i][0]), axis=None)
    

print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (2680, 150)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [7]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2412, 1, 150)
(2412, 4)
(268, 1, 150)
(268, 4)


In [8]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [9]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#3: Force Merged",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-09 04:47:51,536] A new study created in RDB with name: Case#3: Force Merged
[I 2023-10-09 04:47:52,641] Trial 0 finished with value: 1.0 and parameters: {'lr': 0.0010964549524828137, 'dropout_prob': 0.46112021000191755, 'hidden_channels': 40, 'n_layers': 3, 'kernel_size': 8, 'stride': 6, 'padding': 5, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 0 with value: 1.0.


Trial failed with error: Given input size: (40x1x1). Calculated output size: (40x1x0). Output size is too small


[I 2023-10-09 04:47:53,537] Trial 1 finished with value: 1.0 and parameters: {'lr': 4.797642831959844e-05, 'dropout_prob': 0.1309842737255531, 'hidden_channels': 152, 'n_layers': 6, 'kernel_size': 8, 'stride': 3, 'padding': 1, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 0 with value: 1.0.


Trial failed with error: Calculated padded input size per channel: (5). Kernel size: (8). Kernel size can't be greater than actual input size
Epoch 10/50, Training Loss: 0.5922164618968964, Validation Loss: 0.5909198522567749
Epoch 20/50, Training Loss: 0.3501712622181062, Validation Loss: 0.342895045876503
Epoch 30/50, Training Loss: 0.30493964687470465, Validation Loss: 0.28010986000299454
Epoch 40/50, Training Loss: 0.26274382779675143, Validation Loss: 0.2599780857563019


[I 2023-10-09 04:49:54,011] Trial 2 finished with value: 0.21936751529574394 and parameters: {'lr': 1.6966978060123207e-05, 'dropout_prob': 0.10444288167130766, 'hidden_channels': 144, 'n_layers': 7, 'kernel_size': 7, 'stride': 1, 'padding': 10, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 2 with value: 0.21936751529574394.


...

Epoch 50/50, Training Loss: 1.156720510710241e-05, Validation Loss: 0.0010669447069062699
Epoch 10/50, Training Loss: 0.004113026694188105, Validation Loss: 0.01793076370813651
Epoch 20/50, Training Loss: 0.0016528185947062147, Validation Loss: 0.014423150912080018
Epoch 30/50, Training Loss: 3.3052750437021926e-05, Validation Loss: 0.010157334298810383
Epoch 40/50, Training Loss: 1.46918855314711e-05, Validation Loss: 0.010880053265282186


[I 2023-10-09 06:08:21,728] Trial 98 finished with value: 0.011240609463584406 and parameters: {'lr': 0.0029247061499720435, 'dropout_prob': 0.15850865197672087, 'hidden_channels': 220, 'n_layers': 4, 'kernel_size': 8, 'stride': 7, 'padding': 10, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 34 with value: 0.0004163870631828104.


Epoch 50/50, Training Loss: 9.004657818701573e-06, Validation Loss: 0.011240609463584406
Epoch 10/50, Training Loss: 0.006608782421549662, Validation Loss: 0.003228861194656929
Epoch 20/50, Training Loss: 0.006299887292815613, Validation Loss: 0.002619928160129348
Epoch 30/50, Training Loss: 0.004576442233032609, Validation Loss: 0.0007070340852806112
Epoch 40/50, Training Loss: 0.0027577193634662706, Validation Loss: 0.0015778911993038491


[I 2023-10-09 06:09:33,802] Trial 99 finished with value: 0.005285043177991611 and parameters: {'lr': 0.0024314274465931612, 'dropout_prob': 0.17701541884786703, 'hidden_channels': 180, 'n_layers': 4, 'kernel_size': 9, 'stride': 7, 'padding': 9, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 34 with value: 0.0004163870631828104.


Epoch 50/50, Training Loss: 0.0046614365547288945, Validation Loss: 0.005285043177991611


In [10]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.006780637977518086, 'dropout_prob': 0.2192170613629412, 'hidden_channels': 240, 'n_layers': 3, 'kernel_size': 8, 'stride': 7, 'padding': 9, 'pooling_type': 'avg', 'use_batch_norm': False}
Best final validation loss:
0.0004163870631828104


In [11]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/force_merged_best_configs.json"

In [13]:

# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [12]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You